In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 11,padding="same") #   RF = 11*11
        self.norm1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)  # RF = 22*22
        self.conv2 = nn.Conv2d(32, 64, 1, padding="same") 
        self.norm2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 1, padding="same")
        self.avgpool=  nn.AvgPool2d(7)
        self.fc    = nn.Linear(128,10)

        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x =  self.dropout(self.pool1(self.norm1(F.relu(self.conv1(x)))))
        x = self.dropout(self.pool2(self.norm2(F.relu(self.conv2(x)))))
        x = self.avgpool(F.relu(self.conv3(x)))
        x = x.view(-1, 128)
        x = self.fc(x)

        return F.log_softmax(x, dim = 1)

In [4]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu") 
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]           3,904
       BatchNorm2d-2           [-1, 32, 28, 28]              64
         MaxPool2d-3           [-1, 32, 14, 14]               0
           Dropout-4           [-1, 32, 14, 14]               0
            Conv2d-5           [-1, 64, 14, 14]           2,112
       BatchNorm2d-6           [-1, 64, 14, 14]             128
         MaxPool2d-7             [-1, 64, 7, 7]               0
           Dropout-8             [-1, 64, 7, 7]               0
            Conv2d-9            [-1, 128, 7, 7]           8,320
        AvgPool2d-10            [-1, 128, 1, 1]               0
           Linear-11                   [-1, 10]           1,290
Total params: 15,818
Trainable params: 15,818
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

In [5]:


torch.manual_seed(1)
batch_size = 256

kwargs = {'num_workers': 6, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [12]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [13]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.95)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.11994815617799759 batch_id=234: 100%|█| 235/235 [00:01<00:00, 146.31it/s]



Test set: Average loss: 0.1656, Accuracy: 9504/10000 (95%)



loss=0.08065957576036453 batch_id=234: 100%|█| 235/235 [00:01<00:00, 147.69it/s]



Test set: Average loss: 0.1201, Accuracy: 9635/10000 (96%)



loss=0.11442121118307114 batch_id=234: 100%|█| 235/235 [00:01<00:00, 148.01it/s]


...


Test set: Average loss: 0.0384, Accuracy: 9882/10000 (99%)



loss=0.005960257723927498 batch_id=234: 100%|█| 235/235 [00:01<00:00, 145.99it/s



Test set: Average loss: 0.0491, Accuracy: 9860/10000 (99%)



loss=0.03694431856274605 batch_id=234: 100%|█| 235/235 [00:01<00:00, 146.65it/s]



Test set: Average loss: 0.0369, Accuracy: 9885/10000 (99%)

